In [ ]:
import pandas as pd
import numpy as np

# Load dataset

# Store original columns

def CORRELATED(df1):
    X=df1.drop('Label', axis=1)
    original_columns = set(X.columns)
       
    ### 1. Remove features with zero standard deviation ###
    zero_std_features = [col for col in X.columns if X[col].std() == 0]
    X = X.drop(columns=zero_std_features)
    print("Removed Zero Std Features:", zero_std_features)
    
    ### 2. Remove highly correlated features (keep first occurrence) ###
    correlation_matrix = X.corr().abs()
    upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    
    # Identify highly correlated feature pairs (keep first occurrence)
    highly_correlated_features = set()
    for column in upper_triangle.columns:
        correlated_cols = upper_triangle.index[upper_triangle[column] == 1].tolist()
        for col in correlated_cols:
            if col not in highly_correlated_features:  # Keep the first occurrence
                highly_correlated_features.add(column)
    
    X = X.drop(columns=highly_correlated_features)
    print("Removed Highly Correlated Features (keeping first occurrence):", highly_correlated_features)
    
    ### 3. Remove redundant features (identical columns) ###
    duplicate_features = X.T.duplicated()
    redundant_features = X.columns[duplicate_features].tolist()
    X = X.loc[:, ~duplicate_features]
    print("Removed Redundant Features:", redundant_features)
    X['Label']=df1['Label']
    return X
file_path = r"D:\RAGHU\DATASET\CSV_DATASET\4L_INBOUND_DATASET\Original\REPLACE_NEGATIVE\CICDDoS2019_77F_TRAIN_DATASET_FRONT_SEQUANTIAL_MULTI_CLASS_with_imputation_4L.csv"
df = pd.read_csv(file_path, low_memory=False)

df = CORRELATED(df)
### 4. Remove duplicate rows ###


Removed Zero Std Features: ['Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'FIN Flag Count', 'PSH Flag Count', 'ECE Flag Count', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']
Removed Highly Correlated Features (keeping first occurrence): {'Subflow Fwd Bytes', 'Subflow Bwd Bytes', 'Subflow Fwd Packets', 'Subflow Bwd Packets', 'Avg Fwd Segment Size', 'RST Flag Count'}
Removed Redundant Features: []


In [6]:
before_rows = df.shape[0]
df = df.drop_duplicates()
after_rows = df.shape[0]
removed_rows = before_rows - after_rows
print(f"Removed Duplicate Rows: {removed_rows}")

# Save the cleaned dataset
cleaned_file_path = r"D:\RAGHU\Cleaned_Dataset.csv"
df.to_csv(cleaned_file_path, index=False)

# Summary
# removed_features = list(set(original_columns) - set(df.columns))
# print(f"Total Features Removed: {len(removed_features)}")
# print(f"Final Dataset Shape: {df.shape}")
# print(f"Cleaned dataset saved at: {cleaned_file_path}")
df

Removed Duplicate Rows: 0


,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound,Label
0,110861755,26,26,616.0,768.0,297.0,0.0,23.692308,80.460062,384.0,...,198805.454545,64223.338986,392446.0,179351.0,9.863210e+06,413171.813769,10007500.0,8632089.0,0,BENIGN
1,40335006,9,10,8.0,62.0,1.0,0.0,0.888889,0.333333,31.0,...,90226.500000,43.554563,90287.0,90185.0,9.993447e+06,40495.753715,10018634.0,9933709.0,0,BENIGN
2,113244633,56,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,248176.750000,859690.585416,2978061.0,4.0,9.188876e+06,809901.667647,9882838.0,6781893.0,0,BENIGN
3,95628949,21,20,20.0,0.0,1.0,0.0,0.952381,0.218218,0.0,...,14901.222222,177.422362,15367.0,14798.0,1.001418e+07,5184.077926,10016037.0,10000366.0,0,BENIGN
4,95613243,21,20,20.0,0.0,1.0,0.0,0.952381,0.218218,0.0,...,26508.333333,17.902514,26534.0,26473.0,1.000431e+07,21.430119,10004365.0,10004289.0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399991,221068,6,0,2088.0,0.0,393.0,321.0,348.000000,35.088460,0.0,...,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,1,UDP-lag
399993,105882,4,0,1398.0,0.0,369.0,330.0,349.500000,22.516660,0.0,...,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,1,UDP-lag
399994,216908,6,0,2088.0,0.0,393.0,321.0,348.000000,35.088460,0.0,...,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,1,UDP-lag
399996,216232,6,0,2088.0,0.0,393.0,321.0,348.000000,35.088460,0.0,...,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,1,UDP-lag


In [14]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare

def first_digit(series):
    """Extracts the first non-zero digit of each number in a series, including decimals starting with 0."""
    def get_first_non_zero_digit(number):
        # Convert the number to a string and remove any non-numeric characters (e.g., commas)
        num_str = str(number).replace(',', '')
        # Iterate through each character in the string
        for char in num_str:
            if char.isdigit() and char != '0':
                return int(char)
        return np.nan  # Return NaN if no non-zero digit is found
    
    return series.apply(get_first_non_zero_digit).dropna().astype(int)

# Manually defined Benford's Law probabilities for digits 1-9
BENFORD_PROBS = np.array([0.301, 0.176, 0.125, 0.097, 0.079, 0.067, 0.058, 0.051, 0.046])

def chi_square_test(observed_counts, expected_counts):
    """Performs a Chi-Square test comparing observed and expected values."""
    return np.sum(((observed_counts - expected_counts) ** 2) / expected_counts)

def apply_benford_law(df):
    """Applies Benford's Law to each numerical column and calculates Chi-Square."""
    chi_stats = []  # List to store Chi-Square statistics for each column
    observed_counts_all = []  # List to store observed counts for each column
    processed_columns = []  # List to store names of columns that were processed
    
    for column in df.select_dtypes(include=[np.number]).columns:
        # Extract the first non-zero digit from the column
        first_digits = first_digit(df[column].dropna())
        if first_digits.empty:
            continue  # Skip columns with no valid first digits
        
        total_values = len(first_digits)
        observed_counts = [0] * 9  # Initialize list for counts of first digits (1-9)
        
        # Count occurrences of each first digit (1-9)
        for digit in range(1, 10):
            observed_counts[digit - 1] = (first_digits == digit).sum()
        
        # Store observed counts for the current column
        observed_counts_all.append(observed_counts)
        
        # Calculate the expected counts based on Benford's Law probabilities
        expected_counts = BENFORD_PROBS * total_values
        
        # Calculate Chi-Square statistic for the column
        chi_square_statistic = chi_square_test(np.array(observed_counts), expected_counts)
        
        # Store the Chi-Square statistic for the column
        chi_stats.append(chi_square_statistic)
        processed_columns.append(column)  # Track processed columns
    
    # Convert Chi-Square results to a DataFrame
    chi_stats_df = pd.DataFrame(processed_columns, columns=['Features'])
    chi_stats_df['Chi-Square']=chi_stats
    #chi_stats_df = pd.DataFrame([[processed_columns,chi_stats]], columns=['Features','Chi-Square'], index=processed_columns)
    
    # Convert observed counts to a DataFrame for easy visualization
    observed_counts_df = pd.DataFrame(observed_counts_all, columns=[f'Digit {i}' for i in range(1, 10)], 
                                      index=processed_columns)
    
    return chi_stats_df, observed_counts_df

# Load dataset
# file_path = r"D:\RAGHU\DATASET\CSV_DATASET\4L_INBOUND_DATASET\Original\REPLACE_NEGATIVE\CICDDoS2019_77F_TRAIN_DATASET_FRONT_SEQUANTIAL_MULTI_CLASS_with_imputation_4L.csv"
# df = pd.read_csv(file_path, low_memory=False)

# Apply Benford's Law and get Chi-Square values and observed counts for each numerical column
chi_stats_df, observed_counts_df = apply_benford_law(df)

# Display results
print("\nChi-Square Statistics for Each Feature:\n", chi_stats_df)
print("\nObserved Counts for Each Digit (1-9) for Each Feature:\n", observed_counts_df)

# Define a threshold for Chi-Square statistics (adjust as necessary)



Chi-Square Statistics for Each Feature:
                        Features    Chi-Square
0                 Flow Duration  3.212001e+04
1             Total Fwd Packets  8.651470e+04
2        Total Backward Packets  1.168111e+05
3   Total Length of Fwd Packets  5.577674e+04
4   Total Length of Bwd Packets  7.221144e+03
5         Fwd Packet Length Max  2.601599e+05
6         Fwd Packet Length Min  1.614359e+05
7        Fwd Packet Length Mean  2.133738e+05
8         Fwd Packet Length Std  5.649365e+04
9         Bwd Packet Length Max  3.623191e+03
10        Bwd Packet Length Min  8.260893e+03
11       Bwd Packet Length Mean  5.059084e+03
12        Bwd Packet Length Std  1.869371e+03
13                 Flow Bytes/s  2.566669e+04
14               Flow Packets/s  1.242954e+04
15                Flow IAT Mean  2.697106e+04
16                 Flow IAT Std  3.977463e+04
17                 Flow IAT Max  2.390297e+04
18                 Flow IAT Min  7.542877e+04
19                Fwd IAT Total  2.461

In [18]:
Mean=chi_stats_df[chi_stats_df['Chi-Square'] >= chi_stats_df['Chi-Square'].mean()]
Features=Mean['Features'].tolist()

In [20]:
Features

['Fwd Packet Length Max',
 'Fwd Packet Length Min',
 'Fwd Packet Length Mean',
 'Bwd Header Length',
 'Min Packet Length',
 'Max Packet Length',
 'Packet Length Mean',
 'Average Packet Size',
 'Init_Win_bytes_forward',
 'Init_Win_bytes_backward',
 'min_seg_size_forward',
 'Inbound']

In [20]:
chi_stats_df['Chi-Square'].median()

32906.210806457275

In [22]:
Median=chi_stats_df[chi_stats_df['Chi-Square'] >= chi_stats_df['Chi-Square'].median()]
len(Median)

30

In [24]:
Union_df = pd.concat([Median, Mean]).drop_duplicates()
Union_df = Union_df.reset_index(drop=True)

print(Union_df)

                       Features    Chi-Square
0             Total Fwd Packets  8.651470e+04
1        Total Backward Packets  1.168111e+05
2   Total Length of Fwd Packets  5.577674e+04
3         Fwd Packet Length Max  2.601599e+05
4         Fwd Packet Length Min  1.614359e+05
5        Fwd Packet Length Mean  2.133738e+05
6         Fwd Packet Length Std  5.649365e+04
7                  Flow IAT Std  3.977463e+04
8                  Flow IAT Min  7.542877e+04
9                  Fwd IAT Mean  3.461791e+04
10                  Fwd IAT Std  5.188990e+04
11                  Fwd IAT Min  6.559070e+04
12                  Bwd IAT Min  3.290367e+04
13            Fwd Header Length  1.031401e+05
14            Bwd Header Length  1.456173e+05
15                Fwd Packets/s  3.788182e+04
16                Bwd Packets/s  8.737279e+04
17            Min Packet Length  1.662866e+05
18            Max Packet Length  1.671525e+05
19           Packet Length Mean  1.626621e+05
20            Packet Length Std  4

In [26]:
Intersection_df = pd.merge(Median, Mean)

# Reset index if needed
Intersection_df = Intersection_df.reset_index(drop=True)

print(Intersection_df)

                   Features    Chi-Square
0     Fwd Packet Length Max  2.601599e+05
1     Fwd Packet Length Min  1.614359e+05
2    Fwd Packet Length Mean  2.133738e+05
3         Bwd Header Length  1.456173e+05
4         Min Packet Length  1.662866e+05
5         Max Packet Length  1.671525e+05
6        Packet Length Mean  1.626621e+05
7       Average Packet Size  3.154927e+05
8    Init_Win_bytes_forward  1.469525e+06
9   Init_Win_bytes_backward  2.056994e+06
10     min_seg_size_forward  2.523465e+05
11                  Inbound  2.678052e+05


In [38]:
Features = Intersection_df['Features'].tolist()
Features.append('Label')
Features

['Fwd Packet Length Max',
 'Fwd Packet Length Min',
 'Fwd Packet Length Mean',
 'Bwd Header Length',
 'Min Packet Length',
 'Max Packet Length',
 'Packet Length Mean',
 'Average Packet Size',
 'Init_Win_bytes_forward',
 'Init_Win_bytes_backward',
 'min_seg_size_forward',
 'Inbound',
 'Label']

In [40]:
Median

,Features,Chi-Square
1,Total Fwd Packets,8.651470e+04
2,Total Backward Packets,1.168111e+05
3,Total Length of Fwd Packets,5.577674e+04
5,Fwd Packet Length Max,2.601599e+05
6,Fwd Packet Length Min,1.614359e+05
7,Fwd Packet Length Mean,2.133738e+05
8,Fwd Packet Length Std,5.649365e+04
16,Flow IAT Std,3.977463e+04
18,Flow IAT Min,7.542877e+04
20,Fwd IAT Mean,3.461791e+04


In [42]:
Features = Median['Features'].tolist()
Features.append('Label')
Features

['Total Fwd Packets',
 'Total Backward Packets',
 'Total Length of Fwd Packets',
 'Fwd Packet Length Max',
 'Fwd Packet Length Min',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Std',
 'Flow IAT Std',
 'Flow IAT Min',
 'Fwd IAT Mean',
 'Fwd IAT Std',
 'Fwd IAT Min',
 'Bwd IAT Min',
 'Fwd Header Length',
 'Bwd Header Length',
 'Fwd Packets/s',
 'Bwd Packets/s',
 'Min Packet Length',
 'Max Packet Length',
 'Packet Length Mean',
 'Packet Length Std',
 'ACK Flag Count',
 'URG Flag Count',
 'Down/Up Ratio',
 'Average Packet Size',
 'Init_Win_bytes_forward',
 'Init_Win_bytes_backward',
 'act_data_pkt_fwd',
 'min_seg_size_forward',
 'Inbound',
 'Label']